# Redes Neurais

## Backpropagation 

In [6]:
import numpy as np
np.set_printoptions(suppress=True, precision=5)
import math

In [7]:
x = 10
math.e**-x

4.5399929762484875e-05

In [9]:
#suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Função de ativação Sigmoid
def sigmoid(x): 
    try:
        sig =  1 /(1+(math.e**-x))
    except:
        sig = 1
        
    return sig

# Função derivada da função Sigmoid (para cálculo do gradiente)
def deriv_sigmoid(y):
    return y * (1.0 - y)   

In [10]:
# Learning rate
alpha = .1   

In [11]:
# Dataset de Risco de Crédito
import pandas as pd

atributos_pt = [ 'classificação_crédito',  'conta.saldo', 'crédito.duração.meses', 'pagamento.crédito.anterior.status', 'crédito.propósito',
                 'crédito.valor', 'poupança', 'duração.emprego', 'taxa.parcelamento', 'estado_civil', 'fiador', 'residência.duração', 
                 'ativos.atual', 'idade', 'outros.créditos', 'apartamento.tipo', 'banco.crédito', 'ocupação',
                'dependentes', 'telefone', 'trabalhador_estrangeiro']

url = 'https://raw.githubusercontent.com/vladimiralencar/Alunos-UEPB-TopicosEspeciaisEmBancoDeDados/refs/heads/master/RegressaoLogistica/credit_dataset_final.csv'
df = pd.read_csv(url)
df.columns = atributos_pt
print(df.shape)
df.head()

(1000, 21)


,classificação_crédito,conta.saldo,crédito.duração.meses,pagamento.crédito.anterior.status,crédito.propósito,crédito.valor,poupança,duração.emprego,taxa.parcelamento,estado_civil,...,residência.duração,ativos.atual,idade,outros.créditos,apartamento.tipo,banco.crédito,ocupação,dependentes,telefone,trabalhador_estrangeiro
0,1,1,18,3,2,1049,1,1,4,1,...,4,2,21,2,1,1,3,1,1,1
1,1,1,9,3,4,2799,1,2,2,3,...,2,1,36,2,1,2,3,2,1,1
2,1,2,12,2,4,841,2,3,2,1,...,4,1,23,2,1,1,2,1,1,1
3,1,1,12,3,4,2122,1,2,3,3,...,2,1,39,2,1,2,2,2,1,2
4,1,1,12,3,4,2171,1,2,4,3,...,4,2,38,1,2,2,2,1,1,2


In [12]:
X = df.iloc[:, 1:].values

y = df['classificação_crédito']. values

In [13]:
X[:3]

array([[   1,   18,    3,    2, 1049,    1,    1,    4,    1,    1,    4,
           2,   21,    2,    1,    1,    3,    1,    1,    1],
       [   1,    9,    3,    4, 2799,    1,    2,    2,    3,    1,    2,
           1,   36,    2,    1,    2,    3,    2,    1,    1],
       [   2,   12,    2,    4,  841,    2,    3,    2,    1,    1,    4,
           1,   23,    2,    1,    1,    2,    1,    1,    1]])

In [14]:
X.shape

(1000, 20)

In [15]:
y[:3]

array([1, 1, 1])

In [16]:
2*np.random.random((3,4))

array([[1.79737, 1.34304, 1.27266, 0.70007],
       [0.7885 , 0.26215, 1.29525, 0.73926],
       [0.0649 , 1.29031, 0.96625, 0.54076]])

In [17]:
# Inicializando randomicamente os vetores de pesos (serão criadas 2 camadas ocultas)
np.random.seed(1)
theta0 = 2*np.random.random((3,4)) - 1
theta1 = 2*np.random.random((4,1)) - 1

In [18]:
theta0

array([[-0.16596,  0.44065, -0.99977, -0.39533],
       [-0.70649, -0.81532, -0.62748, -0.30888],
       [-0.20647,  0.07763, -0.16161,  0.37044]])

In [19]:
theta1

array([[-0.5911 ],
       [ 0.75623],
       [-0.94522],
       [ 0.34094]])

In [20]:
import matplotlib.pyplot as plt

def plotg(erros):
    plt.xlabel('Época')
    plt.ylabel('Erro')
    plt.title("Treinamento do Modelo")
    plt.plot(erros)
    
# plotg(erros)

### Passos Realizados pelo Backpropagation

1- Feedforward pass: Inicializamos aleatoriamente os vetores de peso e multiplicamos a entrada com os vetores de peso subsequentes em direção a uma saída final.

2- Cálculo do erro: Calculamos o erro / perda da saída do passo de feedforward.

3- Backpropagation para a última camada oculta (em relação à saída): Calculamos o gradiente desse erro e alteramos os pesos para a direção do gradiente. Fazemos isso multiplicando o vetor de pesos pelos gradientes calculados.

4- Atualiza os pesos até o critério de parada ser atingido (erro mínimo ou número de rodadas de treinamento).

In [21]:
X.shape

(1000, 20)

In [23]:
input_layer = X.copy()
input_layer.shape

(1000, 20)

In [49]:
import sys

# input_layer = X
# theta0 = 2*np.random.random((3,4)) - 1
# theta1 = 2*np.random.random((4,1)) - 1
# l1 = sigmoid(np.dot(input_layer, theta0)) # camada 1
# l2 = sigmoid(np.dot(l1,theta1))           # camada 2

def train(X,y, learning_rate=.1):

    # Learning rate
    alpha = learning_rate  
    
    # Inicializando randomicamente os vetores de pesos (serão criadas 2 camadas ocultas)
    np.random.seed(1)
    # theta0 = 2*np.random.random((3,4)) - 1
    # theta1 = 2*np.random.random((4,1)) - 1
    
    theta0 = 2*np.random.random((20,20)) - 1
    theta1 = 2*np.random.random((20,1)) - 1
    
    # Loop for para percorrer a rede neural
    # O valor 205000 especifica a quantidade de rounds de treinamento
    epoch = 120 #205000
    erros = []
    for iter in range(epoch): 
        # Etapa 1: Feedforward 
        input_layer = X
        l1 = sigmoid(np.dot(input_layer, theta0)) # camada 1
        l2 = sigmoid(np.dot(l1,theta1))           # camada 2
    
        # Etapa 2: Calculando o erro 
        l2_error = y - l2
        erro =  np.sum(l2_error**2) / l2_error.shape[0] 
        erros.append(erro) # * 1000)
        
        # if (iter% 1) == 0:
        acurácia = np.mean(1-(np.abs(l2_error)))
        sys.stderr.write('\rÉpoca: ' + str(iter) + '/' + str(epoch)+ '- Acurácia da Rede Neural: ' + \
                             str(np.mean(1-(np.abs(l2_error)))))
        sys.stderr.flush()
            
        # Etapa 3: Calculando os gradientes de forma vetorizada 
        l2_delta = alpha * (l2_error * deriv_sigmoid(l2)) # alfa = taxa de aprendizagem
        l1_error = l2_delta.T.dot(theta1)
        l1_delta = alpha * (l1_error * deriv_sigmoid(l1))
    
        # Etapa 4 - Atualiza os vetores de pesos
        theta1 += l1.T.dot(l2_delta)
        theta0 += input_layer.T.dot(l1_delta)
        
    acuracia = np.mean(1-(np.abs(l2_error)))
    print ("Acurácia Final da Rede Neural: " + str(acuracia))
    return acuracia, erros
    
acuracia, erros = train(X, y)
plotg(erros)

Época: 0/120- Acurácia da Rede Neural: 0.6272842714089475

ValueError: shapes (1000,1000) and (20,1) not aligned: 1000 (dim 1) != 20 (dim 0)

In [30]:
l2

array([[0.00153, 0.01665, 0.     , ..., 0.5108 , 0.45953, 0.00002],
       [0.00133, 0.02878, 0.00001, ..., 0.17057, 0.21835, 0.00002],
       [0.00211, 0.01751, 0.00001, ..., 0.25125, 0.40819, 0.00001],
       ...,
       [0.00429, 0.00639, 0.00002, ..., 0.26126, 0.4759 , 0.00008],
       [0.00446, 0.00674, 0.00001, ..., 0.14352, 0.54539, 0.00006],
       [0.00409, 0.00463, 0.00001, ..., 0.21109, 0.68896, 0.00005]])

In [31]:
input_layer = X
l1 = sigmoid(np.dot(input_layer, theta0)) # camada 1
l2 = sigmoid(np.dot(l1,theta1))           # camada 2
l2

array([[0.00153, 0.01665, 0.     , ..., 0.5108 , 0.45953, 0.00002],
       [0.00133, 0.02878, 0.00001, ..., 0.17057, 0.21835, 0.00002],
       [0.00211, 0.01751, 0.00001, ..., 0.25125, 0.40819, 0.00001],
       ...,
       [0.00429, 0.00639, 0.00002, ..., 0.26126, 0.4759 , 0.00008],
       [0.00446, 0.00674, 0.00001, ..., 0.14352, 0.54539, 0.00006],
       [0.00409, 0.00463, 0.00001, ..., 0.21109, 0.68896, 0.00005]])

In [32]:
x1 = np.array([   1,   18,    3,    2, 1049,    1,    1,    4,    1,    1,    4,
           2,   21,    2,    1,    1,    3,    1,    1,    1])

In [34]:
input_layer = x1
l1 = sigmoid(np.dot(input_layer, theta0)) # camada 1
l2 = sigmoid(np.dot(l1,theta1))           # camada 2
l2.shape

(1000,)

---

# Colocar os dados em Escala

In [ ]:
for c in range(X.shape[1]):
    X_normalized[:][c] = ( X[:][c]-X[:][c].mean())/X[:][c].std()
X_normalized[:3]

In [ ]:
%%time
acuracia, erros = train(X_normalized, y, 0.1)
plotg(erros)